In [69]:
import hilbert

x, y = hilbert.genHilbert(4, 4)

In [70]:
print(x)
print()
print(y)

[0, 1, 1, 0, 0, 0, 1, 1, 2, 2, 3, 3, 3, 2, 2, 3]

[0, 0, 1, 1, 2, 3, 3, 2, 2, 3, 3, 2, 1, 1, 0, 0]


In [71]:
def genPoint(offsetX,offsetY):
    for i in range(16):
        x[i]=(x[i]+offsetX)%4
    for i in range(16):
        y[i]=(y[i]+offsetY)%4
    return x,y

In [72]:
x,y=genPoint(2,1)
print(x)
print()
print(y)

[2, 3, 3, 2, 2, 2, 3, 3, 0, 0, 1, 1, 1, 0, 0, 1]

[1, 1, 2, 2, 3, 0, 0, 3, 3, 0, 0, 3, 2, 2, 1, 1]
